# EDA Clínico con Sweetviz (NHANES)

Este notebook realiza un análisis exploratorio de datos (EDA) sobre el dataset clínico de NHANES, validando la calidad y distribución de los nuevos biomarcadores.

In [ ]:
# Instalación de dependencias si es necesario
# !pip install sweetviz pandas pyarrow matplotlib seaborn

In [ ]:
import pandas as pd
import sweetviz as sv
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
pd.set_option('display.max_columns', None)
%matplotlib inline

## 1. Carga y Preparación de Datos

In [ ]:
DATA_PATH = "../data/02_intermediate/process_data.parquet"

try:
    df = pd.read_parquet(DATA_PATH)
    print(f"Datos cargados correctamente. Dimensiones: {df.shape}")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo en {DATA_PATH}")
    # Fallback para desarrollo si el path cambia
    # df = pd.read_parquet("../data/processed/NHANES_ULTIMATE_CLEAN.parquet")

In [ ]:
# Mapeo de columnas a nombres en inglés (Todas las variables disponibles)
rename_dict = {
    'Sexo': 'Sex',
    'Edad': 'Age',
    'Raza': 'Race',
    'Educacion': 'Education',
    'Ingresos_Ratio': 'IncomeRatio',
    'TARGET': 'HeartDisease',
    'Presion_Sistolica': 'SystolicBP',
    'BMI': 'BMI',
    'Cintura': 'WaistCircumference',
    'Altura': 'Height',
    'Colesterol_Total': 'TotalCholesterol',
    'Trigliceridos': 'Triglycerides',
    'LDL': 'LDL',
    'HbA1c': 'HbA1c',
    'Glucosa': 'Glucose',
    'Creatinina': 'Creatinine',
    'Acido_Urico': 'UricAcid',
    'Enzima_ALT': 'ALT',
    'Albumina': 'Albumin',
    'Potasio': 'Potassium',
    'Sodio': 'Sodium',
    'Enzima_GGT': 'GGT',
    'Enzima_AST': 'AST',
    'Fumador': 'Smoking',
    'Actividad_Fisica': 'PhysicalActivity',
    'Seguro_Medico': 'HealthInsurance'
}

df_english = df.rename(columns=rename_dict)

# Eliminar columnas de identificación si existen (ej. SEQN)
if 'SEQN' in df_english.columns:
    df_english = df_english.drop(columns=['SEQN'])

# Asegurar que el Target sea binario (0/1)
if 'HeartDisease' in df_english.columns:
    df_english['HeartDisease'] = df_english['HeartDisease'].astype(int)

# Usar todas las columnas disponibles para el análisis (Sin filtrado)
# El objetivo es tener una visión holística
df_analysis = df_english.copy()

print(f"Columnas para análisis (Total: {len(df_analysis.columns)}):")
print(df_analysis.columns.tolist())

# Mantenimiento de listas para compatibilidad con celdas posteriores del notebook

# Detectar automáticamente variables numéricas para la matriz de correlación
numeric_cols = df_analysis.select_dtypes(include=['number']).columns.tolist()
if 'HeartDisease' in numeric_cols:
    numeric_cols.remove('HeartDisease')

# 'biomarkers' se usa en la celda siguiente para .describe()
# Ahora incluye todas las variables numéricas (biomarcadores + demografía numérica etc.)
biomarkers = numeric_cols

# 'categorical_vars' para referencia
categorical_vars = df_analysis.select_dtypes(exclude=['number']).columns.tolist()
if 'HeartDisease' not in categorical_vars and 'HeartDisease' in df_analysis.columns:
    categorical_vars.append('HeartDisease')


## 2. Validación de Outliers y Estadísticas Descriptivas

Revisamos rápidamente rangos imposibles (ej. Glucosa 0, Presión > 300) antes de generar el reporte visual.

In [ ]:
# Estadísticas descriptivas de biomarcadores numéricos
numeric_cols = [c for c in biomarkers if c in df_analysis.columns]
desc = df_analysis[numeric_cols].describe()
print(desc)

# Chequeo rápido de valores nulos
print("\nValores Nulos:")
print(df_analysis.isnull().sum())

## 3. Análisis de Correlación (Pearson)

Buscamos colinealidad entre variables numéricas.

In [ ]:
plt.figure(figsize=(12, 10))
corr_matrix = df_analysis[numeric_cols].corr()
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Matriz de Correlación de Biomarcadores")
plt.show()

## 4. Reporte Sweetviz (Target: HeartDisease)

Generamos el reporte comparativo para visualizar la separación entre pacientes sanos y con enfermedad cardíaca.

In [ ]:
target_col = 'HeartDisease'

if target_col in df_analysis.columns:
    # Configuración de Sweetviz
    # pairwise_analysis='off' para eficiencia si hay muchas columnas, aunque aquí hemos filtrado.
    report = sv.analyze(
        df_analysis, 
        target_feat=target_col, 
        pairwise_analysis='off'
    )
    
    output_file = "NHANES_Clinical_Analysis.html"
    report.show_html(output_file)
    print(f"Reporte generado exitosamente: {output_file}")
else:
    print(f"Error: La variable objetivo '{target_col}' no se encuentra en el dataframe.")